In [3]:
import pandas as pd
import os
import geopandas as gpd
import re # regular expression
import numpy as np

In [60]:
# excel spreadsheet copied to data/udot folder from https://docs.google.com/spreadsheets/d/1ETo8X8ViekCuP79-8tL3bDOA1SDZZhIA/
fnExcelAADTHistory = 'inputs/AADTHistory 2023.xlsx'
aadt_unrounded_sheetname = 'UnroundedAADT2023'

# excel spreadsheet copied to data/udot folder from https://docs.google.com/spreadsheets/d/1HNS6uq6QwD4JmvwQR7d7kEkkJ9ban4kPQ284ceUTNiI/edit?gid=0#gid=0
fnCsvTruck2023 = 'inputs/Truck Traffic on Utah Highways 2023.csv'
truck_2023_df = pd.read_csv(fnCsvTruck2023)

# from Natalia on 2/18/2025
fnSegmentsShapefiles = [
    'inputs/Summit-Wasatch/SuWs_Segments_20240312.shp'
]
output_shapefile_fn = 'outputs/shapefile/SuWs_Segments_20240312_with_AADT_through_2023.shp'
output_shapefile_with_trucks_fn = 'outputs/shapefile/SuWs_Segments_20240312_with_AADT_through_2023_with_Trucks.shp'



# Get Segments

In [5]:
# read in segment shapefile

gdfSegments = pd.DataFrame()

for file in fnSegmentsShapefiles:
    print(file)
    _df = gpd.read_file(file)
    print(_df)
    _df['SOURCE'] = os.path.basename(file)
    gdfSegments = pd.concat([gdfSegments, _df])

#display(gdfSegments)

# show columns
print(gdfSegments.columns.tolist())

gdfSegments.to_csv('outputs/segments.gdf')

inputs/Summit-Wasatch/SuWs_Segments_20240312.shp
     Id       SEGID     BMP     EMP  DISTANCE     FULLNAME   FNCLASS  FC_CODE  \
0     0  0032_000.0   0.000   9.014  8.994137       HWY 32  Arterial        4   
1     0  0032_009.0   9.014   9.828  0.813032       HWY 32  Arterial        4   
2     0  0032_009.8   9.828  10.404  0.570644       HWY 32  Arterial        4   
3     0  0032_010.4  10.404  11.416  1.012796       HWY 32  Arterial        4   
4     0  0032_011.4  11.416  12.429  1.014125       HWY 32  Arterial        4   
..   ..         ...     ...     ...       ...          ...       ...      ...   
374   0   UDOT_7067   0.000   0.000  2.040250         None  Arterial       40   
375   0  3131_002.1   2.011   2.031  3.752780  LONGVIEW DR  Arterial        5   
376   0  3131_002.2   2.011   2.031  3.752780  LONGVIEW DR  Arterial        5   
377   0  0222_000.2   0.201   1.563  0.026208      HWY 222  Arterial        5   
378   0  3122_000.2   0.190   0.772  0.131334        600 W  

# Get Historic Data from AADTHistory UDOT Spreadsheet

In [6]:
#import excel spreadsheet
xl_file1 = pd.ExcelFile(fnExcelAADTHistory)
dfs1 = {sheet_name: xl_file1.parse(sheet_name) for sheet_name in xl_file1.sheet_names}

In [7]:
#show sheet
display(xl_file1.sheet_names)
dfs1[aadt_unrounded_sheetname]

['RoundedAADT2023', 'UnroundedAADT2023']

,STATION,RouteID,BeginPoint,EndPoint,Section_Length,DESC,AADT2023,AADT2022,AADT2021,AADT2020,...,SUTRK2014,CUTRK2014,SUTRK2013,CUTRK2013,SUTRK2012,CUTRK2012,SUTRK2011,CUTRK2011,SUTRK2010,CUTRK2010
0,027-0005,0006PM,0.000,46.0380,46.0380,Nevada State Line via SR 6 - Antelope Springs,457,441.0,474.0,430.0,...,0.249610,0.232449,0.249610,0.232449,0.249610,0.232449,0.249610,0.232449,0.239372,0.103931
1,027-0007,0006PM,46.038,77.5560,31.5180,Antelope Springs via SR 6 - Gunnison Massacre ...,409,395.0,424.0,385.0,...,0.175063,0.333819,0.175063,0.333819,0.175063,0.333819,0.175063,0.333819,0.175063,0.333819
2,027-0010,0006PM,77.556,82.8970,5.3410,Gunnison Massacre Site via SR 6 - Main St Hin...,586,566.0,609.0,552.0,...,0.162481,0.264722,0.162481,0.264722,0.162481,0.264722,0.162481,0.264722,0.162481,0.264722
3,027-0015,0006PM,82.897,83.9110,1.0140,Main St via SR 6 (500 N) Hinckley - SR 257 (40...,2189,2113.0,2272.0,2061.0,...,0.149898,0.195620,0.149898,0.195620,0.149898,0.195620,0.149898,0.195620,0.149898,0.195620
4,027-0025,0006PM,83.911,87.6940,3.7830,SR 257 (4000 W) Hinckley via SR 6 - 1000 W to ...,4012,3910.0,3852.0,3409.0,...,0.137314,0.126513,0.137314,0.126513,0.137314,0.126513,0.137314,0.126513,0.137314,0.126513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4562,057-1157,3468PM,0.000,5.1248,5.1248,SR 39 Huntsville via 7800 E - 2200 North Eden,2851,2733.0,2764.0,2578.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4563,057-1255,3469PM,0.000,6.9300,6.9300,Snow Basin via Old Snow Basin Rd - SR 39,387,371.0,375.0,350.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4564,057-1460,3470PM,0.000,1.0394,1.0394,2700 N (SR 134) via 1000 W - Pleasant View Dr ...,1710,1686.0,1669.0,1540.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4565,057-1195,3478PM,0.000,2.0404,2.0404,Entrance Bluff Swim Beach via 1st St - SR 39 (...,1006,965.0,976.0,910.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [8]:
_df = dfs1[aadt_unrounded_sheetname]


# get columns with AADT at beginning of name
aadt_columns = [
    col for col in _df.columns 
    if col.startswith('AADT') 
       and len(col) == 8 
       and col[-4:].isdigit()
]

# get dataframe for just segids and aadt columns
_df = _df[['STATION','RouteID','BeginPoint','EndPoint'] + aadt_columns]

# melt to make long
_df = _df.melt(id_vars=('STATION','RouteID','BeginPoint','EndPoint'), var_name='YEAR', value_name='AADT')

# filter out zero volumes
_df = _df[_df['AADT']>0]

# get YEAR from string
_df['YEAR'] = _df['YEAR'].str[-4:].astype(int)

# set AADT to int
_df['AADT'] = _df['AADT'].astype(int)

dfAadtFromExcel = _df
dfAadtFromExcel

,STATION,RouteID,BeginPoint,EndPoint,YEAR,AADT
0,027-0005,0006PM,0.0000,46.0380,2023,457
1,027-0007,0006PM,46.0380,77.5560,2023,409
2,027-0010,0006PM,77.5560,82.8970,2023,586
3,027-0015,0006PM,82.8970,83.9110,2023,2189
4,027-0025,0006PM,83.9110,87.6940,2023,4012
...,...,...,...,...,...,...
195591,049-0720,2899PM,0.7740,1.2210,1981,2925
195592,049-0721,2899PM,1.2210,2.4271,1981,2925
195593,049-0725,2899PM,2.4271,4.1070,1981,2635
195594,049-0726,2899PM,4.1070,4.8110,1981,850


In [9]:
# combine 2022, 2021 and under 2021
dfAADT = pd.concat([dfAadtFromExcel])

dfAADT['ROUTE_ID'] = dfAADT['RouteID']
dfAADT['FROM_MEASURE'] = dfAADT['BeginPoint'].round(2)
dfAADT['TO_MEASURE'] = dfAADT['EndPoint'].round(2)

# melt for easier joining
dfAADT = pd.pivot_table(dfAADT,index=('ROUTE_ID','FROM_MEASURE','TO_MEASURE'),columns='YEAR',values='AADT')

dfAADT.reset_index(inplace=True)
dfAADT.fillna('0',inplace=True)

# Filter rows with 'ROUTE_ID' containing 'PM' - Positive Direction & Mainline?
dfAADT_filtered = dfAADT[dfAADT['ROUTE_ID'].str.contains('PM')].copy()

# Trim 'ROUTE_ID' down to the first four characters in order to match with SEGID
dfAADT_filtered['ROUTE_ID'] = dfAADT_filtered['ROUTE_ID'].str[:4]

dfAADT_filtered

C:\Users\bhereth\AppData\Local\Temp\ipykernel_27320\3334563436.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  dfAADT.fillna('0',inplace=True)


YEAR,ROUTE_ID,FROM_MEASURE,TO_MEASURE,1981,1982,1983,1984,1985,1986,1987,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,0006,0.00,46.04,325.0,335.0,430.0,580.0,585.0,585.0,595.0,...,350.0,375.0,399.0,409.0,412.0,415.0,430.0,474.0,441.0,457.0
1,0006,46.04,77.56,325.0,335.0,430.0,580.0,585.0,585.0,595.0,...,360.0,390.0,412.0,366.0,369.0,372.0,385.0,424.0,395.0,409.0
2,0006,77.56,82.90,520.0,535.0,630.0,780.0,790.0,790.0,800.0,...,450.0,480.0,512.0,525.0,529.0,533.0,552.0,609.0,566.0,586.0
3,0006,82.90,83.91,0,1350.0,1450.0,1650.0,1675.0,1600.0,1525.0,...,1675.0,1800.0,1913.0,1961.0,1975.0,1991.0,2061.0,2272.0,2113.0,2189.0
4,0006,83.91,87.69,0,2650.0,2750.0,3000.0,3050.0,2920.0,2790.0,...,3495.0,3660.0,3846.0,3468.0,3496.0,3573.0,3409.0,3852.0,3910.0,4012.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4562,3468,0.00,5.12,0,0,0,0,0,0,0,...,2120.0,2245.0,2354.0,2452.0,2550.0,2581.0,2578.0,2764.0,2733.0,2851.0
4563,3469,0.00,6.93,0,0,0,0,0,0,0,...,515.0,545.0,572.0,333.0,346.0,350.0,350.0,375.0,371.0,387.0
4564,3470,0.00,1.04,0,0,0,0,0,0,0,...,0,0,0,1653.0,1694.0,1724.0,1540.0,1669.0,1686.0,1710.0
4565,3478,0.00,2.04,0,0,0,0,0,0,0,...,1110.0,1175.0,1232.0,865.0,900.0,911.0,910.0,976.0,965.0,1006.0


In [10]:
# CHECK FOR DUPLICATES

# Define the columns you want to check for duplicates
columns_to_check = ['ROUTE_ID', 'FROM_MEASURE', 'TO_MEASURE']

# Use the duplicated method to find duplicates in those columns
duplicates = dfAADT_filtered.duplicated(subset=columns_to_check, keep=False)

# Filter the DataFrame to only include the duplicates
dfAADT_duplicates = dfAADT_filtered[duplicates]

display(dfAADT_duplicates)

YEAR,ROUTE_ID,FROM_MEASURE,TO_MEASURE,1981,1982,1983,1984,1985,1986,1987,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023


In [11]:
# JOIN DATA BACK TO SEGMENTS FILE AND FILTER WITH MIDPOINT_MP BETWEEN BMP AND EMP

_df = gdfSegments


# calculate midpoint for 
_df['MIDPOINT_MP'] = (_df['BMP'] + _df['EMP']) / 2
_df['ROUTE_ID'] = _df['SEGID'].str.split('_').str[0]

# get only data needed to join
_df = _df[['SEGID','ROUTE_ID','MIDPOINT_MP']]

# create temp dataframe to merge to segments data
_df2 = pd.DataFrame.merge(_df, dfAADT_filtered, on='ROUTE_ID')

# filter for only segments with midpoints between AADT FROM_MEASURE and TO_MEASURE
_df2 = _df2[_df2['MIDPOINT_MP'].between(_df2['FROM_MEASURE'],_df2['TO_MEASURE'])].copy()

# give me only columns with years
cols_to_keep = [col for col in _df2.columns if str(col).isdigit() and 1981 <= int(col) <= 3000]

# filter by only segid and columns with years
_df2 = _df2[['SEGID']+cols_to_keep]

# melt to make table long
_df2 = _df2.melt(id_vars=('SEGID'),var_name="YEAR",value_name='AADT')

# convert AADT to int
_df2['AADT'] = _df2['AADT'].astype(int)

# add source
_df2['SOURCE'] = fnExcelAADTHistory.split('/')[-1]

# filter out an zero data
_df2 = _df2[_df2['AADT']>0]

aadt_from_historic_df = _df2

aadt_from_historic_df

,SEGID,YEAR,AADT,SOURCE
0,0032_000.0,1981,1425,AADTHistory 2023.xlsx
1,0032_009.0,1981,1425,AADTHistory 2023.xlsx
2,0032_009.8,1981,1490,AADTHistory 2023.xlsx
3,0032_010.4,1981,1530,AADTHistory 2023.xlsx
4,0032_011.4,1981,1530,AADTHistory 2023.xlsx
...,...,...,...,...
15045,3136_001.0,2023,512,AADTHistory 2023.xlsx
15046,3131_002.1,2023,564,AADTHistory 2023.xlsx
15047,3131_002.2,2023,564,AADTHistory 2023.xlsx
15048,0222_000.2,2023,4533,AADTHistory 2023.xlsx


In [12]:
aadt_from_historic_df[aadt_from_historic_df['SEGID']=='0032_009.0']

,SEGID,YEAR,AADT,SOURCE
1,0032_009.0,1981,1425,AADTHistory 2023.xlsx
351,0032_009.0,1982,1470,AADTHistory 2023.xlsx
701,0032_009.0,1983,1485,AADTHistory 2023.xlsx
1051,0032_009.0,1984,1485,AADTHistory 2023.xlsx
1401,0032_009.0,1985,1500,AADTHistory 2023.xlsx
1751,0032_009.0,1986,1520,AADTHistory 2023.xlsx
2101,0032_009.0,1987,1575,AADTHistory 2023.xlsx
2451,0032_009.0,1988,1530,AADTHistory 2023.xlsx
2801,0032_009.0,1989,1550,AADTHistory 2023.xlsx
3151,0032_009.0,1990,700,AADTHistory 2023.xlsx


# Wrap up and export

In [13]:
aadt_from_historic_df['AADTCOLNAME'] = 'AADT' + aadt_from_historic_df['YEAR'].astype(str)
aadt_from_historic_pivot_df = aadt_from_historic_df.pivot_table(index='SEGID',columns='AADTCOLNAME',values='AADT').fillna(0).reset_index()

# Convert all AADT columns to integers after pivoting
aadt_columns = aadt_from_historic_pivot_df.columns.difference(['SEGID'])  # Get all columns except SEGID
aadt_from_historic_pivot_df[aadt_columns] = aadt_from_historic_pivot_df[aadt_columns].astype(int)

aadt_from_historic_pivot_df

AADTCOLNAME,SEGID,AADT1981,AADT1982,AADT1983,AADT1984,AADT1985,AADT1986,AADT1987,AADT1988,AADT1989,...,AADT2014,AADT2015,AADT2016,AADT2017,AADT2018,AADT2019,AADT2020,AADT2021,AADT2022,AADT2023
0,0032_000.0,1425,1470,1485,1485,1500,1520,1575,1530,1550,...,1895,2005,2103,2158,3027,3063,3060,3280,3244,3383
1,0032_009.0,1425,1470,1485,1485,1500,1520,1575,1530,1550,...,1895,2005,2103,2158,3027,3063,3060,3280,3244,3383
2,0032_009.8,1490,1535,1550,1580,1595,1615,1635,1585,1605,...,1970,2085,2188,2245,2335,2363,2361,2531,2503,2611
3,0032_010.4,1530,1580,1595,1620,1635,1650,1665,1615,1635,...,4550,4820,5054,5185,5392,5457,5452,5845,5780,6029
4,0032_011.4,1530,1580,1595,1620,1635,1650,1665,1615,1635,...,4550,4820,5054,5185,5392,5457,5452,5845,5780,6029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,3133_000.0,0,0,0,0,0,0,0,0,0,...,0,0,0,1000,1000,1012,1011,1084,1072,1118
346,3134_000.0,0,0,0,0,0,0,0,0,0,...,0,0,0,1037,1063,1082,966,1047,1058,1073
347,3135_000.0,0,0,0,0,0,0,0,0,0,...,0,0,0,500,500,500,500,500,505,512
348,3136_000.0,0,0,0,0,0,0,0,0,0,...,0,0,0,500,500,500,500,500,505,512


In [16]:
# Identify overlapping columns (excluding 'SEGID')
overlapping_columns = gdfSegments.columns.intersection(aadt_from_historic_pivot_df.columns).difference(['SEGID'])

# Drop overlapping columns from gdfSegments
gdfSegments_cleaned = gdfSegments.drop(columns=overlapping_columns)

# Perform the merge
gdfSegments_merged = gdfSegments_cleaned.merge(aadt_from_historic_pivot_df, on='SEGID', how='left').fillna(0).drop(columns='Id')

# Convert all AADT columns to integers after pivoting
# Use regex to filter column names that match the pattern "AADTYYYY" (where YYYY is a four-digit year)
aadt_columns = [col for col in gdfSegments_merged.columns if re.match(r'^AADT\d{4}$', col)]

gdfSegments_merged[aadt_columns] = gdfSegments_merged[aadt_columns].astype(int)

# Display the result
print(gdfSegments_merged)

          SEGID     BMP     EMP  DISTANCE     FULLNAME   FNCLASS  FC_CODE  \
0    0032_000.0   0.000   9.014  8.994137       HWY 32  Arterial        4   
1    0032_009.0   9.014   9.828  0.813032       HWY 32  Arterial        4   
2    0032_009.8   9.828  10.404  0.570644       HWY 32  Arterial        4   
3    0032_010.4  10.404  11.416  1.012796       HWY 32  Arterial        4   
4    0032_011.4  11.416  12.429  1.014125       HWY 32  Arterial        4   
..          ...     ...     ...       ...          ...       ...      ...   
374   UDOT_7067   0.000   0.000  2.040250            0  Arterial       40   
375  3131_002.1   2.011   2.031  3.752780  LONGVIEW DR  Arterial        5   
376  3131_002.2   2.011   2.031  3.752780  LONGVIEW DR  Arterial        5   
377  0222_000.2   0.201   1.563  0.026208      HWY 222  Arterial        5   
378  3122_000.2   0.190   0.772  0.131334        600 W  Arterial        5   

     FC_GROUP                  FC_NAME  CO_FIPS  ... AADT2014 AADT2015  \
0

In [58]:
_df = gdfSegments_merged
_df3 = truck_2023_df[['ROUTE_ID','FROM_MEASURE','TO_MEASURE','SUTRK2023','CUTRK2023']].copy()

_df3['ROUTE_ID'] = _df3['ROUTE_ID'].str[0:4]

print(_df3)

# calculate midpoint for 
_df['MIDPOINT_MP'] = (_df['BMP'] + _df['EMP']) / 2
_df['ROUTE_ID'] = _df['SEGID'].str.split('_').str[0]

#print(_df[['ROUTE_ID','MIDPOINT_MP']])

# get only data needed to join
# create temp dataframe to merge to segments data
_df2 = pd.DataFrame.merge(_df, _df3, on='ROUTE_ID')

# filter for only segments with midpoints between AADT FROM_MEASURE and TO_MEASURE
_df2 = _df2[_df2['MIDPOINT_MP'].between(_df2['FROM_MEASURE'],_df2['TO_MEASURE'])].copy()


# Identify SEGID values that are in _df but missing in _df2
missing_segids = _df.loc[~_df['SEGID'].isin(_df2['SEGID'])]

# Append these missing rows to _df2
_df2 = pd.concat([_df2, missing_segids], ignore_index=True)

_df2.drop(columns=['MIDPOINT_MP','FROM_MEASURE','TO_MEASURE'], inplace=True)

# Convert percentage strings to numeric values
_df2['SUTRK2023'] = _df2['SUTRK2023'].str.rstrip('%').astype(float) / 100
_df2['CUTRK2023'] = _df2['CUTRK2023'].str.rstrip('%').astype(float) / 100

_df2 = _df2.fillna(0)

print(_df2)

gdfSegments_merged_with_trucks_2023 = _df2

     ROUTE_ID  FROM_MEASURE  TO_MEASURE SUTRK2023 CUTRK2023
0        0006         0.000      46.038    25.00%    23.20%
1        0006        46.038      77.556    17.50%    33.40%
2        0006        77.556      82.897    16.20%    26.50%
3        0006        82.897      83.911    15.00%    19.60%
4        0006        83.911      87.694    13.70%    12.70%
...       ...           ...         ...       ...       ...
1872     0319         0.000       0.901     9.50%    25.50%
1873     0491         0.000       0.456     6.40%    33.10%
1874     0491         0.456       2.031     6.40%    33.10%
1875     0491         2.031      17.069     6.40%    33.10%
1876     089A         0.000       2.970    11.50%     5.70%

[1877 rows x 5 columns]
          SEGID     BMP     EMP  DISTANCE     FULLNAME   FNCLASS  FC_CODE  \
0    0032_000.0   0.000   9.014  8.994137       HWY 32  Arterial        4   
1    0032_009.0   9.014   9.828  0.813032       HWY 32  Arterial        4   
2    0032_009.8   9.828 

In [61]:
# Define the output shapefile path

# Save the GeoDataFrame as a shapefile
gdfSegments_merged.to_file(output_shapefile_fn, driver="ESRI Shapefile")

print(f"Shapefile saved to: {output_shapefile_fn}")

# Define the output shapefile path

# Save the GeoDataFrame as a shapefile
gdfSegments_merged_with_trucks_2023.to_file(output_shapefile_with_trucks_fn, driver="ESRI Shapefile")

print(f"Shapefile saved to: {output_shapefile_with_trucks_fn}")

Shapefile saved to: outputs/shapefile/SuWs_Segments_20240312_with_AADT_through_2023.shp
Shapefile saved to: outputs/shapefile/SuWs_Segments_20240312_with_AADT_through_2023_with_Trucks.shp


C:\Users\bhereth\AppData\Local\Temp\ipykernel_27320\3566139010.py:4: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdfSegments_merged.to_file(output_shapefile_fn, driver="ESRI Shapefile")
c:\Users\bhereth\Anaconda3\envs\march2024\lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'MIDPOINT_MP' to 'MIDPOINT_M'
  ogr_write(
